In [1]:
import re
import os
import pandas as pd

results_dir = '../results/test/'

df = pd.DataFrame()

for root, dirs, files in os.walk(results_dir):
    for F in files:
        file_path = os.path.join(root, F)
        df_file = pd.read_json(file_path)
        # df_file['index'] = range(len(df_file))
        # df_file.set_index('index')
        # display(df_file)
        df = pd.concat([df, df_file])
df['Dataset'] = df['Model'].apply(lambda x: x.split('_')[2])
df['Language'] = df['Model'].apply(lambda x: x.split('_')[3]).apply(lambda x: 'Code' if x == 'code' else 'Natural')
df['Rationale'] = df['Model'].apply(lambda x: x.split('_')[4]).apply(lambda x: 'No' if x == 'base' else 'Yes')
df['Model'] = df['Model'].apply(lambda x: (re.search(r'./models/([\w\-\.]+)_ft', x)).group(1))
# df.drop(['Model'], axis=1, inplace=True)
# cols = df.columns.tolist()
# cols = [cols[-1]] + cols[:-1]
# df = df[cols]
df = df.rename(columns={'F1_Score': 'F1'})
df.sort_values(by=['Model', 'Dataset', 'Language', 'Rationale'], inplace=True, )
df = df.round(3)
df.reset_index(inplace=True)
df.drop(['index', 'date', 'n_icl_samples', 'n_samples_test', 'split', 'schema_path', 'dataset'], axis = 1, inplace=True)
display(df)

,Model,Precision,Recall,F1,Dataset,Language,Rationale
0,CodeQwen1.5-7B,0.782,0.762,0.772,ade,Code,No
1,CodeQwen1.5-7B,0.744,0.686,0.714,ade,Code,Yes
2,CodeQwen1.5-7B,0.743,0.701,0.722,ade,Natural,No
3,CodeQwen1.5-7B,0.774,0.665,0.715,ade,Natural,Yes
4,CodeQwen1.5-7B,0.619,0.590,0.604,conll04,Code,No
...,...,...,...,...,...,...,...
91,deepseek-coder-7b-instruct-v1.5,0.499,0.452,0.474,conll04,Natural,Yes
92,deepseek-coder-7b-instruct-v1.5,0.326,0.335,0.330,scierc,Code,No
93,deepseek-coder-7b-instruct-v1.5,0.269,0.275,0.272,scierc,Code,Yes
94,deepseek-coder-7b-instruct-v1.5,0.351,0.352,0.352,scierc,Natural,No


In [2]:
paper_dir = '/home/pgajo/DeepKE/example/llm/CodeKGC/paper'
table_path = os.path.join(paper_dir, 'results.tex')
df.to_latex(table_path, float_format=lambda x: f'{x:.6g}', index=False)